In [3]:
# from pymongo import MongoClient
import numpy as np
import pandas as pd
import datetime
# %load_ext line_profiler
pd.options.mode.chained_assignment = None
from datetime import timedelta
import sys
sys.path.insert(0, 'C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src')
import dbUnit as db
import computeUnit as cu

192.168.1.31:27017


### NONNAV BREAKUP

In [4]:
from pymongo import MongoClient
client = MongoClient("192.168.1.31:27017")
dbb = client["Returns_WealthNFunds"]

def getDataFromMongo(collectionName, date=""):
    collection = dbb.get_collection(collectionName)
    if collection.count_documents({}) > 0:
        if collectionName == "groupreturns":
            data = pd.DataFrame(list(collection.find({"ASONDATE": date}))).drop(columns="_id")
        else:
            data = pd.DataFrame(list(collection.find())).drop(columns="_id")
    else:
        data = pd.DataFrame()
    return data

In [109]:
date = "2018-11-12"
histData = db.getDataFromMongo("rawdatareturns_nonnavport")

In [43]:
retBalSheet = db.getDataFromMongo("rawdatareturns_nonnavport")

retBalSheet = cu.concatCategory(retBalSheet)

In [44]:
retBalSheet

OPERATION                     ACCRUAL  ADMIN FEES  AMMORTIZATION-ACCRETION  \
ASONDATE   PORTCODE                                                          
2018-11-11 500032                0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
2018-11-12 500032            22044.11         0.0                   107.20   
           500032:Large Cap   5605.75         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap  16438.36         0.0                   107.20   
2018-11-13 500032             1432.74         0.0                   108.09   
           500032:Large Cap     35.48         0.0                     0.00   
           500032:Mid Cap       27.40         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap   1369.86         0.0                   108.09   
2018-11-14 500032             1432.74         0.0                   108.98   
           500032:Large Cap     35.48         0.0                     0.00   
           500032:Mid Cap       27.40         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:Small Cap   1369.86         0.0                   108.98   
2018-11-15 500032              610.83         0.0                   105.47   
           500032:Large Cap     35.48         0.0                     0.00   
           500032:Mid Cap       27.40         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap    547.95         0.0                   105.47   
2018-11-16 500032             1377.95         0.0                   106.33   
           500032:Large Cap     35.48         0.0                     0.00   
           500032:Mid Cap       27.40         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap   1315.07         0.0                   106.33   
2018-11-17 500032             1377.95         0.0                   273.57   
...                               ...         ...                      ...   
2018-11-27 500032                0.00         0.0                     0.00   
           500032:Large Cap      0.00         0.0                     0.00   
           500032:Mid Cap        0.00         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap      0.00         0.0                     0.00   
2018-11-28 500032                0.00         0.0                     0.00   
           500032:Large Cap      0.00         0.0                     0.00   
           500032:Mid Cap        0.00         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap      0.00         0.0                     0.00   
2018-11-29 500032                0.00         0.0                     0.00   
           500032:Large Cap      0.00         0.0                     0.00   
           500032:Mid Cap        0.00         0.0                     0.00   
           500032:NIfty          0.00         0.0                     0.00   
           500032:NONE           0.00         0.0                     0.00   
           500032:Small Cap      0.00         0.0                     0.00   
2018-11-30 500032     

In [35]:
rettest = retBalSheet.reset_index().set_index(["ASONDATE"])

forPort = rettest[~rettest["PORTCODE"].str.contains(":", na=False)].reset_index().set_index(["ASONDATE", "PORTCODE"])
forNonPort = rettest[rettest["PORTCODE"].str.contains(":", na=False)].reset_index().set_index(["ASONDATE", "PORTCODE"])

# netSUM:
netPortSum = forPort.sum(1)
netNonPortSum = forNonPort.drop(["CASH IN-OUT", "MAN FEES", "ADMIN FEES"], 1).sum(1)
netSum = pd.concat([netPortSum, netNonPortSum], axis=1).fillna(0).sum(1)

# GrossSUM:
grPortSum = forPort.sum(1)
grNonPortSum = forNonPort.drop(["CASH IN-OUT"], 1).sum(1)
grSum = pd.concat([grPortSum, grNonPortSum], axis=1).fillna(0).sum(1)

rettest = rettest.reset_index().set_index(["ASONDATE", "PORTCODE"])
rettest["NETSUMXACTION"] = netSum
rettest["GRSUMXACTION"] = grSum

balSheet = rettest

  # Initialize closing balance df
closingBal = pd.DataFrame(index=balSheet.index)
closingBal = closingBal.assign(**{"GROPEN": np.zeros(balSheet.index.size), "NETOPEN": np.zeros(balSheet.index.size), "GRCLOSE": np.zeros(balSheet.index.size), "NETCLOSE": np.zeros(balSheet.index.size), "NETCLWOCASH": np.zeros(balSheet.index.size), "CASHINOUT": balSheet["CASH IN-OUT"], "GRRETURNS": np.zeros(balSheet.index.size), "NETRETURNS": np.zeros(balSheet.index.size), "LOGGRRETURNS": np.zeros(balSheet.index.size), "LOGNETRETURNS": np.zeros(balSheet.index.size)})
closingBal = closingBal.unstack("PORTCODE")

closingBal["GROPEN"].iloc[0] = 0
closingBal["NETOPEN"].iloc[0] = 0

balSheetUnstacked = balSheet.unstack()

In [203]:
balSheetUnstacked.to_csv("unstackBalSheet.csv")

In [36]:
closingBal["NETCLOSE"].values[:] = closingBal["NETOPEN"].values[0] + balSheetUnstacked["NETSUMXACTION"].cumsum().values
closingBal["NETCLWOCASH"] = closingBal["NETCLOSE"].values - closingBal["CASHINOUT"].values

In [37]:
closingBal["GRCLOSE"].values[:] = closingBal["GROPEN"].values[0] + balSheetUnstacked["GRSUMXACTION"].cumsum().values
closingBal["NETOPEN"].values[1:] = closingBal["NETCLOSE"].values[:-1]
closingBal["GROPEN"].values[1:] = closingBal["GRCLOSE"].values[:-1]

In [38]:
closingBal["NETRETURNS"] = closingBal[["NETOPEN", "NETCLWOCASH"]].apply(lambda x: (x['NETCLWOCASH']/x['NETOPEN'])-1, axis=1).replace(np.inf,np.nan).fillna(0)
closingBal["GRRETURNS"] = closingBal["GRCLOSE"].pct_change()    

In [39]:
closingBal["LOGGRRETURNS"] = np.log(closingBal["GRCLOSE"].pct_change() + 1)
closingBal["LOGNETRETURNS"] = np.log(closingBal["NETRETURNS"] + 1)

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [40]:
allDateClosing = pd.DataFrame(index=pd.date_range(closingBal.index.min(), closingBal.index.max()))

In [19]:
closingBal = allDateClosing.merge(closingBal,left_index=True,right_index=True,how="left")
closingBal.columns = pd.MultiIndex.from_tuples(closingBal.columns)
closingBal = closingBal.fillna(method="ffill")

c:\users\admin\appdata\local\programs\python\python37\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)


In [21]:
unstackClosingBal = closingBal.unstack().reset_index().rename(columns={0: "VALUE", "level_0": "OPERATION", "level_1": "PORTCODE", "level_2": "ASONDATE"})

In [42]:
closingBal

GROPEN                                               \
PORTCODE          500027 500027:BSE INDEX   500027:NONE        500032   
ASONDATE                                                                
2018-11-11  0.000000e+00              0.0  0.000000e+00  0.000000e+00   
2018-11-12           NaN              NaN           NaN  7.291000e+09   
2018-11-13           NaN              NaN           NaN  7.291204e+09   
2018-11-14           NaN              NaN           NaN  7.291339e+09   
2018-11-15           NaN              NaN           NaN  7.291428e+09   
2018-11-16           NaN              NaN           NaN  7.286504e+09   
2018-11-17           NaN              NaN           NaN  7.286728e+09   
2018-11-18           NaN              NaN           NaN  7.286672e+09   
2018-11-19           NaN              NaN           NaN  7.286692e+09   
2018-11-20           NaN              NaN           NaN  7.286497e+09   
2018-11-21           NaN              NaN           NaN  7.286574e+09   
2018-11-22           NaN              NaN           NaN  7.286478e+09   
2018-11-23           NaN              NaN           NaN  7.286491e+09   
2018-11-24           NaN              NaN           NaN  7.286493e+09   
2018-11-25           NaN              NaN           NaN  7.286494e+09   
2018-11-26           NaN              NaN           NaN  7.286454e+09   
2018-11-27           NaN              NaN           NaN  7.286449e+09   
2018-11-28           NaN              NaN           NaN  7.286449e+09   
2018-11-29           NaN              NaN           NaN  7.286377e+09   
2018-11-30           NaN              NaN           NaN  7.286377e+09   
2019-04-04           NaN              NaN           NaN  7.286377e+09   
2019-04-05  4.557263e+10        4200000.0  4.556843e+10           NaN   

                                                                     \
PORTCODE   500032:Large Cap 500032:Mid Cap 500032:NIfty 500032:NONE   
ASONDATE                                                              
2018-11-11             0.00           0.00          0.0        0.00   
2018-11-12              NaN            NaN          NaN        0.00   
2018-11-13          2035.48            NaN          NaN        0.00   
2018-11-14          3070.96        6027.40     127228.0        0.00   
2018-11-15          4106.44        7054.80     162228.0         NaN   
2018-11-16          5141.92        8082.20     185328.0        0.00   
2018-11-17          6177.40       10109.60     208428.0    99984.66   
2018-11-18          6712.88       10637.00     225978.0        0.00   
2018-11-19          6948.36       11164.40     229708.0        6.85   
2018-11-20          5283.84       11691.80     129510.0  -100000.00   
2018-11-21          5319.32       11719.20     204510.0  -100000.00   
2018-11-22          7244.80       11496.60     104945.0         NaN   
2018-11-23          6040.28       10834.00     115797.0  -100041.84   
2018-11-24          6075.76       10861.40     115797.0  -100025.03   
2018-11-25          6111.24       10888.80     115797.0  -100008.22   
2018-11-26            79.73       10560.03     115797.0  -101907.25   
2018-11-27            79.73       10560.03     110585.0  -101907.25   
2018-11-28            79.73       10560.03     110585.0  -101907.25   
2018-11-29            79.73       10560.03      38717.0  -101907.25   
2018-11-30            79.73       10560.03      38717.0  -101907.25   
2019-04-04            79.73       10560.03      38717.0  -101907.25   
2019-04-05              NaN            NaN          NaN         NaN   

                                  NETOPEN  ...     LOGGRRETURNS LOGNETRETURNS  \
PORTCODE   500032:Small Cap        500027  ... 500032:Small Cap        500027   
ASONDATE                                   ...                                  
2018-11-11             0.00  0.000000e+00  ...              NaN           0.0   
2018-11-12              NaN           NaN  ...              NaN           0.0   
2018-11-13 

In [28]:
closingBal[["LOGGRRETURNS", "LOGNETRETURNS"]][:"2019-04-05"]

LOGGRRETURNS                                             \
                 500027 500027:BSE INDEX 500027:NONE        500032   
2018-11-11          NaN              NaN         NaN           NaN   
2018-11-12          NaN              NaN         NaN  2.791246e-05   
2018-11-13          NaN              NaN         NaN  1.862073e-05   
2018-11-14          NaN              NaN         NaN  1.214334e-05   
2018-11-15          NaN              NaN         NaN -6.754616e-04   
2018-11-16          NaN              NaN         NaN  3.068213e-05   
2018-11-17          NaN              NaN         NaN -7.655474e-06   
2018-11-18          NaN              NaN         NaN  2.680009e-06   
2018-11-19          NaN              NaN         NaN -2.668919e-05   
2018-11-20          NaN              NaN         NaN  1.051961e-05   
2018-11-21          NaN              NaN         NaN -1.314666e-05   
2018-11-22          NaN              NaN         NaN  1.827599e-06   
2018-11-23          NaN              NaN         NaN  1.913788e-07   
2018-11-24          NaN              NaN         NaN  1.913787e-07   
2018-11-25          NaN              NaN         NaN -5.464149e-06   
2018-11-26          NaN              NaN         NaN -7.153001e-07   
2018-11-27          NaN              NaN         NaN  0.000000e+00   
2018-11-28          NaN              NaN         NaN -9.863289e-06   
2018-11-29          NaN              NaN         NaN  0.000000e+00   
2018-11-30          NaN              NaN         NaN  0.000000e+00   
2018-12-01          NaN              NaN         NaN  0.000000e+00   
2018-12-02          NaN              NaN         NaN  0.000000e+00   
2018-12-03          NaN              NaN         NaN  0.000000e+00   
2018-12-04          NaN              NaN         NaN  0.000000e+00   
2018-12-05          NaN              NaN         NaN  0.000000e+00   
2018-12-06          NaN              NaN         NaN  0.000000e+00   
2018-12-07          NaN              NaN         NaN  0.000000e+00   
2018-12-08          NaN              NaN         NaN  0.000000e+00   
2018-12-09          NaN              NaN         NaN  0.000000e+00   
2018-12-10          NaN              NaN         NaN  0.000000e+00   
...                 ...              ...         ...           ...   
2019-03-07          NaN              NaN         NaN  0.000000e+00   
2019-03-08          NaN              NaN         NaN  0.000000e+00   
2019-03-09          NaN              NaN         NaN  0.000000e+00   
2019-03-10          NaN              NaN         NaN  0.000000e+00   
2019-03-11          NaN              NaN         NaN  0.000000e+00   
2019-03-12          NaN              NaN         NaN  0.000000e+00   
2019-03-13          NaN              NaN         NaN  0.000000e+00   
2019-03-14          NaN              NaN         NaN  0.000000e+00   
2019-03-15          NaN              NaN         NaN  0.000000e+00   
2019-03-16          NaN              NaN         NaN  0.000000e+00   
2019-03-17          NaN              NaN         NaN  0.000000e+00   
2019-03-18          NaN              NaN         NaN  0.000000e+00   
2019-03-19          NaN              NaN         NaN  0.000000e+00   
2019-03-20          NaN              NaN         NaN  0.000000e+00   
2019-03-21          NaN              NaN         NaN  0.000000e+00   
2019-03-22          NaN              NaN         NaN  0.000000e+00   
2019-03-23          NaN              NaN         NaN  0.000000e+00   
2019-03-24          NaN              NaN         NaN  0.000000e+00   
2019-03-25          NaN              NaN         NaN  0.000000e+00   
2019-03-26          NaN              NaN         NaN  0.000000e+00   
2019-03-27          NaN              NaN         NaN  0.000000e+00   
2019-03-28          NaN              NaN         NaN  0.000000e+00   
2019-03-29          NaN              NaN         NaN  0.000000e+00   
2019-03-30          NaN              NaN         NaN  0.000000e+00   
2019-03-31          N

In [24]:
cu.calcGroupReturns('2019-04-05', closingBal, ["LOGGRRETURNS", "LOGNETRETURNS"])

C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src\computeUnit.py:90: RuntimeWarning: invalid value encountered in power
  "2Y_ANN": closingBal[cols][:date].resample("2Y").sum().iloc[-2:].sum().values ** (365/(closingBal[cols][:date].resample("2Y").size().sum())),
C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src\computeUnit.py:92: RuntimeWarning: invalid value encountered in power
  "3Y_ANN": closingBal[cols][:date].resample("3Y").sum().iloc[-2:].sum().values ** (365/(closingBal[cols][:date].resample("3Y").size().sum())),
C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src\computeUnit.py:94: RuntimeWarning: invalid value encountered in power
  "5Y_ANN": closingBal[cols][:date].resample("5Y").sum().iloc[-2:].sum().values ** (365/(closingBal[cols][:date].resample("5Y").size().sum())),
C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src\computeUnit.py:96: RuntimeWarning: invalid value encountered in po

1D   1W  1FN   1M   3M        6M        9M  \
LOGGRRETURNS  500027            0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500027:BSE INDEX  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500027:NONE       0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500032            0.0  0.0  0.0  0.0  0.0 -0.000634 -0.000634   
              500032:Large Cap  0.0  0.0  0.0  0.0  0.0 -0.960830 -0.960830   
              500032:Mid Cap    0.0  0.0  0.0  0.0  0.0  0.752004  0.752004   
              500032:NIfty      0.0  0.0  0.0  0.0  0.0 -0.695688 -0.695688   
              500032:NONE       0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500032:Small Cap  0.0  0.0  0.0  0.0  0.0  1.133579  1.133579   
LOGNETRETURNS 500027            0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500027:BSE INDEX  0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500027:NONE       0.0  0.0  0.0  0.0  0.0  0.000000  0.000000   
              500032            0.0  0.0  0.0  0.0  0.0  0.000052  0.000052   
              500032:Large Cap  0.0  0.0  0.0  0.0  0.0 -0.960830 -0.960830   
              500032:Mid Cap    0.0  0.0  0.0  0.0  0.0  0.752004  0.752004   
              500032:NIfty      0.0  0.0  0.0  0.0  0.0 -0.695688 -0.695688   
              500032:NONE       0.0  0.0  0.0  0.0  0.0 -1.000000 -1.000000   
              500032:Small Cap  0.0  0.0  0.0  0.0  0.0  1.133579  1.133579   

                                      1Y  YTD        2Y        2Y_ANN  \
LOGGRRETURNS  500027            0.000000  0.0  0.000000  0.000000e+00   
              500027:BSE INDEX  0.000000  0.0  0.000000  0.000000e+00   
              500027:NONE       0.000000  0.0  0.000000  0.000000e+00   
              500032           -0.000634  0.0 -0.000634           NaN   
              500032:Large Cap -0.960830  0.0 -0.960830           NaN   
              500032:Mid Cap    0.752004  0.0  0.752004  2.655087e-01   
              500032:NIfty     -0.695688  0.0 -0.695688           NaN   
              500032:NONE       0.000000  0.0  0.000000  0.000000e+00   
              500032:Small Cap  1.133579  0.0  1.133579  6.485651e-01   
LOGNETRETURNS 500027            0.000000  0.0  0.000000  0.000000e+00   
              500027:BSE INDEX  0.000000  0.0  0.000000  0.000000e+00   
              500027:NONE       0.000000  0.0  0.000000  0.000000e+00   
              500032            0.000052  0.0  0.000052  1.925393e-11   
              500032:Large Cap -0.960830  0.0 -0.960830           NaN   
              500032:Mid Cap    0.752004  0.0  0.752004  2.655087e-01   
              500032:NIfty     -0.695688  0.0 -0.695688           NaN   
              500032:NONE      -1.000000  0.0 -1.000000           inf   
              500032:Small Cap  1.133579  0.0  1.133579  6.485651e-01   

                                      3Y        3Y_ANN        5Y  \
LOGGRRETURNS  500027            0.000000  0.000000e+00  0.000000   
              500027:BSE INDEX  0.000000  0.000000e+00  0.000000   
              500027:NONE       0.000000  0.000000e+00  0.000000   
              500032           -0.000634           NaN -0.000634   
              500032:Large Cap -0.960830           NaN -0.960830   
              500032:Mid Cap    0.752004  2.655087e-01  0.752004   
              500032:NIfty     -0.695688           NaN -0.695688   
              500032:NONE       0.000000  0.000000e+00  0.000000   
              500032:Small Cap  1.133579  6.485651e-01  1.133579   
LOGNETRETURNS 500027            0.000000  0.000000e+00  0.000000   
              500027:BSE INDEX  0.000000  0.000000e+00  0.000000   
              500027:NONE       0.000000  0.000000e+00  0.000000   
              500032            0.000052  1.925393e-11  0.000052   
              500032:Large Cap -0.960830           NaN -0.960830   
              500032:Mid Cap    0.752004  2.655087e-01  0.752004   
              500032:NIfty     -0.695688           NaN 

In [29]:
closingBalance = cu.calcDailyCloseReturns(stagingConcat)

C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src\computeUnit.py:181: RuntimeWarning: invalid value encountered in log
  closingBal["LOGNETRETURNS"] = np.log(closingBal["NETRETURNS"] + 1)


In [ ]:
date = '2018-11-14'

In [241]:
date = pd.to_datetime(date)

In [246]:
date.strftime("%Y-%m-%d")

'2018-11-14'

In [240]:
datetime.datetime.strptime(pd.to_datetime(date).isoformat()+".000Z", "%Y-%m-%dT%H:%M:%S.000Z")

datetime.datetime(2018, 11, 14, 0, 0)

In [233]:
datetime.datetime(2015, 3, 1)

datetime.datetime(2015, 3, 1, 0, 0)

In [58]:
testClose = pd.read_csv("C:/Users/admin/mukesh/projects/ideal6-python-worker/iModules/returns/Src/excel/output/Testingclosing.csv")

testClose = testClose.set_index('DATE')

testClose.index = pd.to_datetime(testClose.index)

testClose.resample("1Y").sum().iloc[-2:].sum().values ** (365/(testClose.resample("1Y").size().sum())) 

(365/(testClose.resample("1Y").size().sum()))

testClose.resample('BYS').sum().iloc[-1].values

In [32]:
date="2018-11-14"
closingBal[["LOGGRRETURNS", "LOGNETRETURNS"]][:date].resample('1D').sum().iloc[-2:].sum()

LOGGRRETURNS   500032              0.000028
               500032:Large Cap    0.000000
               500032:NONE              inf
               500032:Small Cap    0.000000
LOGNETRETURNS  500032              0.000028
               500032:Large Cap    0.000000
               500032:NONE              inf
               500032:Small Cap    0.000000
dtype: float64

In [33]:
cu.calcGroupReturns(date, closingBal, ["LOGGRRETURNS", "LOGNETRETURNS"])

1D        1W       1FN        1M  \
LOGGRRETURNS  500032            0.000028  0.000028  0.000028  0.000028   
              500032:Large Cap  0.000000  0.000000  0.000000  0.000000   
              500032:NONE            inf       inf       inf       inf   
              500032:Small Cap  0.000000  0.000000  0.000000  0.000000   
LOGNETRETURNS 500032            0.000028  0.000028  0.000028  0.000028   
              500032:Large Cap  0.000000  0.000000  0.000000  0.000000   
              500032:NONE            inf       inf       inf       inf   
              500032:Small Cap  0.000000  0.000000  0.000000  0.000000   

                                      3M        6M        9M        1Y  \
LOGGRRETURNS  500032            0.000028  0.000028  0.000028  0.000028   
              500032:Large Cap  0.000000  0.000000  0.000000  0.000000   
              500032:NONE            inf       inf       inf       inf   
              500032:Small Cap  0.000000  0.000000  0.000000  0.000000   
LOGNETRETURNS 500032            0.000028  0.000028  0.000028  0.000028   
              500032:Large Cap  0.000000  0.000000  0.000000  0.000000   
              500032:NONE            inf       inf       inf       inf   
              500032:Small Cap  0.000000  0.000000  0.000000  0.000000   

                                      2Y        3Y        5Y  INCEPTION  
LOGGRRETURNS  500032            0.000028  0.000028  0.000028   0.000028  
              500032:Large Cap  0.000000  0.000000  0.000000   0.000000  
              500032:NONE            inf       inf       inf        inf  
              500032:Small Cap  0.000000  0.000000  0.000000   0.000000  
LOGNETRETURNS 500032            0.000028  0.000028  0.000028   0.000028  
              500032:Large Cap  0.000000  0.000000  0.000000   0.000000  
              500032:NONE            inf       inf       inf        inf  
              500032:Small Cap  0.000000  0.000000  0.000000   0.000000

### BENCHMARK

In [6]:
histData = db.getDataFromMongo("rawdatareturns_benchmark")
if not histData.empty:
    print("notEmpty")
    # drop a day records
    histData = histData.set_index("ASONDATE").stack().reset_index().rename(columns={0: "VALUE", "level_1": "BENCHMARK"})
    histData = histData[histData["ASONDATE"] != "2018-04-04"]
    # get daily record
    dailyData = db.getDataFromOracle("trindex_daily", "2018-04-04")
    if not dailyData.empty:
        rawBench = histData.append(dailyData, ignore_index=True)
    else:
        rawBench = histData
else:
    print("Empty")
    histData = db.getDataFromOracle("trindex_all")
    # Save all historical data into Mongo
    rawBench = histData
rawBench.BENCHMARK = rawBench.BENCHMARK.astype(str)
rawBench = rawBench.pivot_table(index="ASONDATE", columns="BENCHMARK", values="VALUE").reset_index()
db.insertDataToMongo("rawdatareturns_benchmark", rawBench)
rawBench = rawBench.set_index("ASONDATE").stack().reset_index().rename(columns={0: "VALUE"})

notEmpty


No data found for trindex_daily inside oracle for 2018-04-04


In [8]:
allDateRawBench = pd.DataFrame(index=pd.date_range(rawBench['ASONDATE'].min(),rawBench['ASONDATE'].max()))
pivotRawBench = rawBench.pivot_table(index="ASONDATE",columns='BENCHMARK',values="VALUE")

In [10]:
rawPrices = allDateRawBench.merge(pivotRawBench,left_index=True,right_index=True,how='left').fillna(method='ffill')
dividend = pd.DataFrame(index=rawPrices.index,columns=rawPrices.columns,data=0)
splitRatio = pd.DataFrame(index=rawPrices.index,columns=rawPrices.columns,data=1)

In [13]:
adjPrice = cu.calcAdjustedPrices(rawPrices,dividend,splitRatio).sort_index()
db.insertDataToMongo("dailyadjclose_benchmark",adjPrice.reset_index())

In [6]:
logReturns = np.log(adjPrice.pct_change() + 1).replace([np.inf, -np.inf], np.nan).fillna(0)
db.insertDataToMongo("dailyadjclose_benchmark",logReturns.reset_index())

In [15]:
adjPrice = adjPrice.stack().reset_index().rename(columns={'level_0':"ASONDATE","level_1":"BENCHMARK",0:"ADJPRICES"})
logReturns = logReturns.stack().reset_index().rename(columns={'level_0':"ASONDATE","level_1":"BENCHMARK",0:"LOGRETURNS"})

NameError: name 'logReturns' is not defined

In [20]:
dailyclreturns_benchmark = adjPrice.merge(logReturns,on=['ASONDATE','BENCHMARK'])
dailyclreturns_benchmark = dailyclreturns_benchmark.set_index(['ASONDATE','BENCHMARK']).stack().reset_index().rename(columns={"level_2":"OPERATION",0:"VALUE"})

In [14]:
# db.insertDataToMongo("dailyclreturns_benchmark",dailyclreturns_benchmark)

In [22]:
dailyclreturns_benchmark = dailyclreturns_benchmark.pivot_table(index='ASONDATE',columns=["OPERATION","BENCHMARK"])
dailyclreturns_benchmark = dailyclreturns_benchmark.iloc[:, dailyclreturns_benchmark.columns.get_level_values(1)=='LOGRETURNS'].fillna(0)

In [24]:
dailyclreturns_benchmark.columns = dailyclreturns_benchmark.columns.droplevel(0)

In [25]:
groupReturns = cu.calcGroupReturns("2018-01-01",dailyclreturns_benchmark,["LOGRETURNS"])
groupReturns['TYPE']="BENCHMARK"
groupReturns['ASONDATE']="2018-01-01"

In [13]:
db.insertDataToMongo("groupreturns",groupReturns.reset_index(),"2018-01-01")

In [14]:
db.insertDataToOracle("groupreturns",groupReturns.reset_index(),type="BENCHMARK")

In [15]:
import pyfolio as pyf

In [16]:
dailyclreturns_benchmark.columns = dailyclreturns_benchmark.columns.droplevel(0)
groupRatio = dailyclreturns_benchmark.apply(lambda x: pyf.timeseries.perf_stats(x))
db.insertDataToMongo("groupratio_navport",groupRatio)

c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:704: RuntimeWarning: invalid value encountered in true_divide
  out=out,
c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:790: RuntimeWarning: invalid value encountered in true_divide
  np.divide(average_annual_return, annualized_downside_risk, out=out)
c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:1511: RuntimeWarning: invalid value encountered in double_scalars
  np.abs(np.percentile(returns, 5))
c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:438: RuntimeWarning: invalid value encountered in double_scalars
  return ending_value ** (1 / num_years) - 1
c:\users\mukesh\appdata\local\programs\python\python37\lib\site-packages\empyrical\stats.py:1477: RuntimeWarning: invalid value encountered in log1p
  cum_log_returns = np.log1p(returns).cumsum()
c:\users\mukesh\app

In [10]:
prodReturns = pd.read_csv('../../portOptHrp/Src/excel/output/dailyPortfolioSumLogReturns_highRiskFund.csv',index_col=0)

In [43]:
productReturns = pd.DataFrame(data={"porduct":[-0.004979822193,0.001275256583,0.01383176591,0.07746582181,0.05906101205,-0.003181742739,0.02885775774,-0.08120523434,0.04087102417,-0.02687326544,0.07288368845,0.08670717845,-0.01159222877,0.002912457945,0.05658951675,0.0149660224,-0.004989044223,0.0006893647075,0.02905849937,0.008491429375,-0.02934357499,-0.04002572741,-0.001281249262,-0.04935673056,0.01359329498,0.02344041107,0.04355343997,-0.08714051769,0.03384320295,0.04103452427,-0.007387039452,-0.0172930426,0.04339208653,0.04077652753,-0.07865351525,-0.005151959926,0.03645489656,0.00447842627,-0.02230700527,-0.04931524434,-0.03739151465,-0.03511735081,0.0349702392,-0.03406412925,0.0289012775,-0.0003472385471,0.01729337415,-0.009082923693,0.07470847871,-0.01542076205,0.008880606183,0.04201089206,0.02264578604,0.03025288707,-0.01227748585,-0.009647773566,0.03115908559,0.009802569351,-0.002492689462,-0.08114934993,-0.03724929536,0.01940281607,-0.08668999554,-0.03978093191,-0.08662370484,0.09008311521,-0.05279816101,-0.07949629952,-0.002809374506,-0.004912427424,0.02842992977,-0.01648783155,0.001840830942,0.02784957091,-0.07075395388,0.04931538342,0.002268381619,-0.05114755193,0.07033023545,-0.01361690961,0.005185154956,0.03442684761,0.02438119255,0.06449327727,0.06325539243,0.01520716337,-0.01119110183,-0.008113205668,-0.01119676106,0.003428908699,-0.03582409655,0.01224427708,-0.01054095623,-0.007621829844,0.04749278291,0.06144832446,-0.0504075988,0.001169839682,-0.007566152084,0.007015182864,0.03706925748,0.03960564202,-0.06522159189,-0.01691642061,-0.01188501001,-0.02826237359,0.005179991855,0.06641205061,-0.0120718337,0.01731438349,-0.02966083609,0.05559247883,0.04278122783,-0.06625402584,-0.03960563277,0.04356297674,-0.05838304662,0.02715010953,0.06709484489,-0.003101580304,0.001142733199,0.02922653152,0.06370325365,0.04474310543,-0.03931777883,0.06471213711,0.05221565732,-0.01370176092,0.004458021271,-0.03707866344,0.01421340823,0.06084216447,0.09082983727,0.01623910534,0.02878564528,0.01304734671,-0.01292224081,-0.04594393103,0.03605184115,0.01698560542,0.01874044449,0.007788491834,0.05886778686,0.02947904228,0.01944611724,0.01918553674,-0.01556610831,-0.03616103577,-0.0156524447,0.02480917951,0.02763827084,0.009006525302,0.02562134266,0.04392968942,-0.006300565586,0.05586656455,0.02932524172,0.05085268218,-0.04662447708,-0.01219299449,0.002063640973,-0.00238513263,-0.07215177044,-0.06044942774,0.02052388516,0.01435489109,0.006799976308,-0.02444117772,-0.07799335184,-0.008231867042,0.004643919089,0.05070541296,0.05761242977,-0.07112900483,0.02147275407,0.04773975785,0.04000350242,0.01243301892,-0.03777412997,0.01111844076,0.0439120881,-0.02640963383,-0.04285038883,-0.02668902379,0.01217479967,-0.001146607807,0.05296016605,0.02281481299,0.01878080518,0.02171568329,-0.002453224841,0.08381349759,-0.01063114467,0.01827379932,-0.01000349414,0.02889363836,0.05958035205,0.007436555043,0.04323388451,0.07650255632,0.01831507277,-0.01997935412,0.02905902787,0.0675352249,0.08751358293,-0.06358120101,0.02525853697,-0.0344437316,-0.08566814535,0.0566598407,-0.08076648564,-0.006281389795,0.03821493163,-0.00642577315,0.04499567628,0.04996156709,-0.0227071335,0.01829524595,-0.004836869568,-0.03887257715,-0.0430863403,0.02169841859,0.004895671136,0.07164862309,-0.0142773742,0.005156360625,0.01073634586,-0.03153160946,0.04132385039,0.03663097169,-0.03289404966,0.01976625203,0.09594375469,0.00002694862035,-0.0684220404,-0.009094142423,-0.04012765252,-0.01115926844,-0.02478317016,0.08143572144,-0.07240421644,-0.01307405067,0.01137392691,0.006715203206,0.08721405633,-0.03946472112,0.005913891003,0.02717253614,0.07091106836,-0.08139440008,0.05429673409,0.05329995802,0.05651557579,-0.001701274827,-0.02136018915,-0.02135084435,-0.03744664046,-0.01537786074,0.04585283634,0.05331938265,0.0445852532,-0.04139091051,-0.00568025777,0.02321961499,-0.01954339025,0.008407151602,0.01619188622,0.01325363645,-0.01498882211,0.05982527381,0.03235561562,0.04929366363,-0.02681832198,0.0009771558233,0.02238441728,-0.05100578314,-0.0177680593,-0.03412034778,0.004559493888,-0.06652508164,-0.0334993839,-0.01859061834,-0.05096562056,-0.0152021218,0.0109147599,-0.05216079768,-0.006175323535,0.08269303677,0.08601923172,-0.03416587913,-0.009418277007,-0.01525034353,0.0240917539,-0.01272620773,-0.05094783546,-0.02400827718,-0.001798902076,-0.001488196518,0.07258930859,0.06072636801,0.01842317044,0.02674745757,-0.01334019609,0.04040392512,0.02589739681,0.01056065216,-0.00848406817,-0.007180460223,-0.01099577594,0.05714344768,-0.02744731391,-0.02434212473,0.01524388675,0.0008243705731,-0.05037896143,-0.02184396725,-0.002113602918,0.01341601095,-0.04162823568,0.06381928512,-0.07061900354,0.02246452646,0.01380299033,0.01095342649,-0.005782044027,0.06283467652,-0.009441642626,-0.005347939482,-0.07884216597,-0.05183298198,0.007035970238,0.02094138522,-0.005693981698,-0.04663506735,0.007036381959,0.001027320224,0.03748561808,0.01844261704,-0.05580306096,-0.05147603113,0.01101272766,-0.04612217723,-0.03249335363,0.01316824968,0.03662433799,-0.01926865704,0.03592911844,0.01140722265,-0.01799888636,0.01192246799,-0.02472412211,-0.06767675714,-0.03970446179,0.001676761718,0.06610745984,0.03437746606,0.006471333828,-0.02655328511,-0.04656942943,-0.03225735395,0.04873608664,-0.02491411579,-0.04234681827,-0.01945877304,-0.04719796864]})
benchmark = pd.DataFrame(data={"porduct":[0.004673704088,0.05858316969,0.08963732198,0.006143464111,-0.04398271558,0.001227635682,0.02864699951,0.06388237523,0.01449000561,0.009963897,-0.04089233559,0.07998071702,0.02236579692,0.02964258916,0.001966676932,-0.03213636436,0.05197946307,0.0577373948,0.01660947189,-0.05962369152,-0.07838085707,-0.007255771167,0.04808833131,-0.04210852636,-0.07026223256,-0.01539600525,-0.04799180241,0.05314129291,0.008295216371,0.009375442661,0.05259752829,0.03095211949,-0.007754520885,0.02631967189,0.05359760993,-0.007332056984,-0.07134480323,0.03672710396,0.00481967762,-0.01564893796,0.002783517157,0.01607438045,-0.08453372593,0.01906681278,-0.05171992225,0.001533936535,0.008217236969,0.05557737901,-0.03945947793,0.02561860668,0.02553672067,0.0266268093,-0.0631813325,0.05428928492,-0.007836010765,0.03535043364,0.01027814612,0.03210639324,-0.06257160203,0.05756342538,-0.04447640656,-0.07373067048,-0.004387409892,0.06252975853,-0.007010586541,0.03880012023,0.03806095137,-0.06123200807,0.07088297571,-0.0551359704,0.02538258241,0.01226805031,-0.06237566836,-0.01078048153,0.0462711313,-0.007676611667,0.01624082387,0.01931565101,-0.0764812476,-0.06988771084,-0.0001571308367,0.007492691946,0.002740313832,-0.002816047705,0.02767018668,-0.06044133706,-0.03356317474,-0.06946468586,0.02845373465,-0.01798551664,0.03453822039,-0.06864813149,-0.03286172049,-0.01046782054,0.06529825689,-0.08462347496,-0.02556312508,-0.07957783572,0.008979813461,0.02467424756,-0.0296433408,0.04763069707,0.02741952828,0.06304890452,-0.005607926099,-0.0008649014645,0.01139871209,0.03558232613,0.0384801738,-0.03595458633,-0.08901584362,0.06035574111,-0.06010274736,-0.03811290627,0.03150965111,-0.02177330325,0.0003290776516,-0.002602256163,-0.002393247899,0.002740079981,-0.04215309274,0.08383187117,-0.06282926314,-0.02767660356,-0.05677068812,-0.04118114272,0.02609893778,-0.06390610214,-0.01605680275,-0.0558330543,-0.08369947867,0.01787374272,-0.01506723516,-0.008770821464,-0.03479108135,-0.04234587227,0.02849102237,0.07207692326,0.07405107085,0.01691442967,-0.03010157647,-0.07085773576,0.02904310836,-0.04092249088,-0.03987345428,-0.01114958772,0.002142288874,0.06124382928,0.07508182279,-0.0170855731,0.007677111478,0.05874594401,-0.01024843459,-0.007021771665,-0.06494855388,-0.03113654777,0.008144068546,-0.01646464999,-0.04098075571,-0.0586144662,-0.06487444517,-0.02544346068,0.0002178842742,0.01303379516,0.006143405706,0.03791697898,0.004807010707,-0.03470105276,0.003802162145,-0.06074164975,0.01227319823,-0.007065439655,0.02691367472,0.0008428184006,0.01577437275,-0.04118591975,0.05397345382,-0.03210980826,-0.01406802991,-0.01885451408,-0.07320434511,0.03307104721,-0.01187754228,-0.007447402064,-0.09814608058,-0.01648025244,0.004163854683,-0.01600025309,-0.07780753984,0.04559824147,0.01998695898,0.03243758032,-0.04241922235,-0.001059126114,-0.007259013361,-0.02391006923,-0.03961192465,-0.04096582963,-0.008424147459,0.04445548276,0.003063741308,-0.0001804140935,-0.05418197107,-0.005973720221,-0.03274028809,-0.04401541021,0.007438261182,-0.005012306131,0.03945897611,0.03045233415,0.002951892444,-0.05356935067,0.05672446338,0.06307057895,0.0379391051,-0.008958537545,0.04632319337,-0.02033244238,-0.0553003056,-0.02999382137,0.0166739644,0.02316643215,-0.0623859376,-0.03111036768,-0.01144603513,-0.02286315792,0.008630114111,0.007513438947,0.0006415220829,0.01713666168,-0.02973018338,0.05418303831,-0.01136249626,0.00352706974,0.01201426151,-0.04342452307,-0.0716635623,0.02012059278,-0.06318526479,0.01335682719,0.02774433413,0.05181981679,0.04317605797,0.07849661073,0.004074381857,0.06078205805,-0.0294925712,-0.03991605463,-0.07338245046,-0.0328351641,-0.006070114223,-0.0728177881,0.04873983209,0.01439179848,0.07099040031,0.063418883,-0.0009373459601,-0.02496523237,0.07326188483,0.06143336822,0.02756769549,0.03321536025,-0.03908889895,0.05256521975,-0.04907571166,0.09363463509,-0.03611590828,-0.07265657006,0.04828026758,0.03515297207,0.06183234303,-0.07563413315,-0.004327886811,-0.02040400489,0.009602131629,-0.01804348041,-0.02468211662,-0.06490554779,0.005992459511,0.0453110322,-0.02535254316,0.03934875817,0.013898985,-0.004556632281,-0.03207857824,-0.04100713662,-0.005617417205,-0.01055657957,-0.00835129729,0.0576673408,0.0706139098,-0.01199478892,0.0571415195,-0.0169259065,-0.03328408061,0.00284078807,0.05166465815,-0.03516563697,-0.02435341091,-0.05255276077,-0.09278451718,0.06225419088,-0.01689353331,-0.04105338009,0.03835073412,0.07396198989,-0.02484709875,-0.01793370667,-0.01573501564,-0.09451158441,-0.009227784666,-0.04370677635,0.03358479759,-0.005965614079,0.03641838962,-0.007807577695,0.003493715369,0.002197673464,-0.001861305539,-0.003866776026,-0.0360965867,0.03826589761,0.03665310728,0.02196150328,-0.04966522612,-0.0810245264,0.01602646605,0.06708274164,-0.002736550539,-0.07402969836,-0.02195047672,0.03008446354,0.007521557816,-0.05346990606,0.04122740084,-0.008104308092,0.04813944169,-0.02821329799,0.01294855973,-0.007752249782,-0.0152034626,-0.03280537778,-0.08819018762,-0.01792370157,0.05032005675,0.00527331709,-0.03173857173,-0.06295942233,0.03027240363,-0.0009955115937,-0.0699502897,0.001403047861,0.0127451994,-0.02690387605,0.01015237515,-0.01581336856,0.04316206623,0.03590715009,0.002139634327,-0.04040338739,-0.02408198965,0.03531417744,0.003735927216,-0.04019011545,-0.04187149979]})

In [47]:
productReturns['porduct'].corr(benchmark['porduct'])

-0.038595763798033256

In [12]:
mar = 0.00370454

def downsiderisk(row,mar):
    returns = row.values[0]
    if returns < mar:
        dr = (returns - mar) * (returns - mar)
    else:
        dr = 0
    return dr

date = "2019-01-23"

In [73]:
import datetime
from dateutil.relativedelta import relativedelta
s = '2020-02-20'
dt = datetime.datetime.strptime(s, '%Y-%m-%d')
d = dt.replace(year=dt.year-1, day=dt.day-1) 
# d = datetime.datetime.strptime(s, '%Y-%m-%d') - datetime.timedelta(days=366)
d = d.strftime('%Y-%m-%d')
print(d)

2019-02-19


In [20]:
prodReturns = prodReturns.loc[d:s]

In [ ]:
benchmark = db.getDataFromMongo("dailyadjclose_benchmark")

In [129]:
prodReturns.index = pd.to_datetime(prodReturns.index)

In [134]:
(np.exp(prodReturns.resample("1Y").sum()) - 1).iloc[-1][0]

0.02120298307423707

In [82]:
benchmark = benchmark["CEMENT"]

In [83]:
benchmark = benchmark.loc[d:s]

In [116]:
returns = prodReturns.merge(pd.DataFrame(benchmark),left_index=True,right_index=True,how="left").fillna(method="ffill")

In [121]:
returns["highRiskFund"].corr(returns["CEMENT"])

nan

In [95]:
prodReturns['highRiskFund'].corr((benchmark))

nan

In [52]:
benchmark.var()[0]

0.001720625339461907

In [42]:
prodReturns['highRiskFund'].corr(benchmark)

nan

In [30]:
prodReturns.merge(pd.DataFrame(benchmark),left_index=True,right_index=True,how="left").corr()

,highRiskFund,CRED
highRiskFund,1.0,NaN
CRED,NaN,NaN


In [74]:
prodReturns.std()[0]

0.010390487604274998

In [ ]:
np.sqrt(productReturns.apply(downsiderisk, args=([mar]),axis=1).mean())

In [81]:
import jsonschema

In [193]:
prodDet =  {
        "event":"createProduct",
        "data":{
            "asondate": "2019-01-23",
            "method":"hrp",
            "riskmodels":"semicovariance",
            "securities":["HDB","AXISBANK.NS","IBN","FEDERALBNK.NS","SBIN.NS","VIJAYABANK.NS"],
            "productname":"highRiskFund",
            "description":"",
            "benchmark":"CRED",
            "mar":0.00370454,
            "riskfreerate":0.05,
            "classification":"",
            "objective":"",
            "launchdate":"2019-01-20",
            "watching":"",
            "invested":"",
            "rating":"",
            "riskprofile":[],
            "stylefactors":"",
            "size":"",
            "sector":"",
            "allocation":"",
            "scenerios":[
                {
                    "name":"Scene1",
                    "fdate" : "2016-01-01",
                    "tdate" : "2017-01-01"
                },
                {
                    "name":"Scene2",
                    "fdate" : "2015-01-01",
                    "tdate" : "2016-01-01"
                }
            ],
            "constraints":[]
        }
    }

In [196]:
jsonschema.validate(prodDet,schema)

In [195]:
schema = {
    "type":"object",
    "properties":{
        "event":{"type":"string"},
        "data":{
            "type":"object",
            "properties":{
                "asondate":{"type":"string"},
                "method":{
                    "type":"string",
                    "pattern":"\\b(ef|hrp)\\b"
                },
                "riskmodels":{"type":"string"},
                "securities":{
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "minItems": 1,
                    "uniqueItems": True
                },
                "productname":{"type":"string"},
                "benchmark":{"type":"string"},
                "mar":{
                    "type":"number",
                    "minimum": 0
                },
                "riskfreerate":{
                    "type":"number",
                    "minimum": 0
                },
                "launchdate":{"type":"string"},
                "scenerios":{
                    "type":"array",
                    "items":{
                        "type":"object",
                        "properties":{
                            "name":{"type":"string"},
                            "fdate" : {"type":"string"},
                            "tdate" : {"type":"string"}
                        }
                    }
                }
            },
            "required":["asondate","method","riskmodels","securities","productname","benchmark","mar","riskfreerate","launchdate"]
        }
    },
    "required":["event","data"]
    
}

In [165]:
daily = pd.read_csv('../../portOptHrp/Src/excel/output/portSecDailyLog_highRiskFund.csv',index_col=0,parse_dates=True)

In [168]:
daily.stack().reset_index()

,Date,level_1,0
0,2015-01-01,HDB,0.000000
1,2015-01-01,AXISBANK.NS,0.000000
2,2015-01-01,IBN,0.000000
3,2015-01-01,FEDERALBNK.NS,0.000000
4,2015-01-01,SBIN.NS,0.000000
5,2015-01-01,VIJAYABANK.NS,0.000000
6,2015-01-01,highRiskFund,0.000000
7,2015-01-02,HDB,0.000000
8,2015-01-02,AXISBANK.NS,0.013679
9,2015-01-02,IBN,0.000000


In [79]:
def applyMaxDrawDown(daily):
    maximum = daily.max()
    dateOfMax = (daily[daily == maximum].index).astype(str)[0]
    filterFromMaxToAsonDate =  daily.loc[dateOfMax:]
    minimum = filterFromMaxToAsonDate.min()
    return(maximum-minimum)

In [80]:
daily.apply(applyMaxDrawDown)

HDB              0.000000
AXISBANK.NS      0.143862
IBN              0.000000
FEDERALBNK.NS    0.092128
SBIN.NS          0.000000
VIJAYABANK.NS    0.000000
highRiskFund     0.192522
dtype: float64

In [216]:
daily = daily['AXISBANK.NS']

In [217]:
daily.max()

0.0837011213848831

In [220]:
daily.max() - daily.loc[(daily[daily == daily.max()].index).astype(str)[0]:].min()

0.14386167983733603

In [8]:
daily = daily.stack().reset_index().rename(columns={"level_1":"SCENERIOS",0:"RETURNS"})
daily['BUCKET']="1D"
daily['PRODUCT'] = daily.iloc[-1]["SCENERIOS"]
daily["BENCHMARK"] = "TEST"

In [39]:
db.insertDataToMongo("bucket_returns",daily)

NameError: name 'daily' is not defined

In [224]:
collection=db.db.get_collection("bucket_returns")
collection.delete_many({"PRODUCT":None})

In [227]:
data[data['SCENERIONAME']=="Sim_Scene2"]

,BUCKET,DATE,PRODUCT,RETURNS,SCENERIONAME,SECURITYCODE
12045,1D,2015-01-01,highRiskFund,-0.005446,Sim_Scene2,highRiskFund
12046,1D,2015-01-02,highRiskFund,-0.000066,Sim_Scene2,highRiskFund
12047,1D,2015-01-03,highRiskFund,0.000125,Sim_Scene2,highRiskFund
12048,1D,2015-01-04,highRiskFund,0.000016,Sim_Scene2,highRiskFund
12049,1D,2015-01-05,highRiskFund,0.014920,Sim_Scene2,highRiskFund
12050,1D,2015-01-06,highRiskFund,0.023579,Sim_Scene2,highRiskFund
12051,1D,2015-01-07,highRiskFund,0.009068,Sim_Scene2,highRiskFund
12052,1D,2015-01-08,highRiskFund,-0.014072,Sim_Scene2,highRiskFund
12053,1D,2015-01-09,highRiskFund,0.002877,Sim_Scene2,highRiskFund
12054,1D,2015-01-10,highRiskFund,0.010512,Sim_Scene2,highRiskFund


In [226]:
data = db.getDataFromMongo("bucket_returns")

In [225]:
data[~data.BENCHMARK.isnull()]

AttributeError: 'DataFrame' object has no attribute 'BENCHMARK'

In [45]:
monthGrp = pd.read_csv('../../portOptHrp/Src/excel/output/ratefeed_2019-01-23_1Mgrp.csv',index_col=0)
monthGrp = monthGrp.stack().reset_index().rename(columns={"level_1":"CODE",0:"RETURNS"})
monthGrp['BUCKET']="1M"